In [2]:
import propka

In [ ]:
proka.run(
    pdb_file="7SL6_medoid_cluster_1_AA_renumbered_SSBOND_fix_heavy_re.pdb", 
    output_file="7SL6_medoid_cluster_1_AA_renumbered_SSBOND_fix_heavy_re.out",
    pH=7.4,
    forcefield="amber",
    ion_strength=0.1,
    temperature=298.15,
    n_threads=4,
    verbose=True)

NameError: name 'proka' is not defined